### **Dictionaries:** (stuff that should have been included in Lecture1)

Dictionaries are defined by the following set of rules:

 - it's an ordered (after pyton 3.7, before they were unordered) set of pairs `key:value`
 - elements are accessed by key and not by index
 - keys must be immutable (e.g., boolean, integer, float, tuple, string, not list)
 - dictionaries themselves are mutable - you can add, delete and change elements

In [ ]:
my_dict = {
  "mykey1": "value1",
  "key2": 2,
  "key3": 3.5
}

In [ ]:
print(my_dict)

In [ ]:
print(len(my_dict))

Another way to construct:

In [ ]:
my_dict2 = dict(name = "Cat", age = 177, country = "Neverland")
print(my_dict2)

In [ ]:
my_dict3 = dict([(1,2),(3,4),(5,6)])
print(my_dict3)

In [ ]:
names = ["Maria", "John", "Stefano", "BunchOfPeople"]
ages = [77, 21, 67, [1, 5, 38, 137]]
age_dict = {k: v for k, v in zip(names, ages)}
print(age_dict)

How to access elements:

In [ ]:
print(my_dict2["age"])

In [ ]:
my_dict2.get("name")

In [ ]:
my_dict2.keys()

In [ ]:
my_dict2.values()

In [ ]:
for x, y in my_dict2.items():
    print(x, y) 

copying (just `=` won't work as it's just a "label"): 

In [ ]:
my_dict4=my_dict2.copy()
print(my_dict4)

adding new elements:

In [ ]:
my_dict2["color"]="black"
print(my_dict2)

modifying elements:

In [ ]:
my_dict2["color"]="purple"
print(my_dict2)

deleting elements:

In [ ]:
del my_dict2["color"]
print(my_dict2)

checking if element exists:

In [ ]:
print("color" in my_dict2)
print("name" in my_dict2)

### **Ctypes**

This is better for pure C than for C++, use `pybind` for C++, overwise you need to write too many wrappers

First, you need to create a `C` shared library.

```g++ -fPIC -shared -o libhellotest.so hello.cpp```

where `hello.cpp` constains some functions, for example:

```
#include <iostream>
extern "C" {
    void hello(){
        std::cout<<"Hello, World!"<<std::endl;
    }
}
```

We can't really use "real C++" inside the `extern C`, so no classes, no templates, only functions.

The compiler flags `-fPIC -shared` are needed to create the library object (you have seen it with `pybind` already)

In [1]:
#install ctypes package in conda
import ctypes
 
libObject = ctypes.CDLL('./libhellotest.so')

In [ ]:
libObject.hello()

So what was that output? Some undefined number that the function returned, the actual "Hello, World!" has been printed to the console (where you have started the jyputer-lab).

What if we need to have parameters/return something?

Add 

```
int sum(int a, int b){
  return a+b;
}
```

to the `hello.cpp` file and then recompile it with: 

```g++ -fPIC -shared -o libhellotest2.so hello.cpp```

In [ ]:
libObject2 = ctypes.CDLL('./libhellotest2.so')

In [ ]:
libObject2.hello() #just to see that it still works

In [ ]:
libObject2.sum(2,3)

This works, but the ctypes had to make a guess about the argument types, so it's very dangerous and you need to actually specify them:

In [ ]:
libObject2.sum.argtypes = [ctypes.c_int, ctypes.c_int]
libObject2.sum.restype = ctypes.c_int

In [ ]:
libObject2.sum(2,3)

Working with strings:

unfortunately, you need pure `C` strings for this to work, not `std::string`, so - char arrays. Let's just create a file (this is overcomplicated cause jupyter doesn't print anythng from C printf): 

In [ ]:
%%file strings.cpp

#include <iostream>
extern "C" {
    void print(char* str) {
       std::cout<<str<<std::endl;
    }
}

compile with

```
g++ -fPIC -shared -o libstrings.so strings.cpp
```

In [ ]:
lo = ctypes.CDLL('./libstrings.so')

In [ ]:
lo.argtypes=[ctypes.c_char_p]

In [ ]:
lo.print(b"MEOW")

In [ ]:
cstring ="MEOW"
lo.print(cstring.encode())

Working with arrays:

In [5]:
%%file arrays.cpp

#include <iostream>

extern "C"{
    int* create_array(int N) {
        int* arr = new int[N];
        return arr;
    }

    void do_something_with_array(int* arr,int N){
        for(int i=0;i<N;i++){
            arr[i]=i;
            std::cout<<arr[i];
        }
        std::cout<<std::endl;
    }
    
    void delete_array(int* arr) {
        delete[] arr;
    }
}

Overwriting arrays.cpp


In [2]:
lo = ctypes.CDLL('./libarr.so')

In [6]:
lo.create_array.restype=ctypes.POINTER(ctypes.c_int)
lo.create_array.argtypes=[ctypes.c_int]
lo.do_something_with_array.argtypes=[ctypes.POINTER(ctypes.c_int),ctypes.c_int]
lo.delete_array.argtypes=[ctypes.POINTER(ctypes.c_int)]

In [8]:
arr=lo.create_array(7)
lo.do_something_with_array(arr,7)
lo.delete_array(arr)

0

This is obviously a huge memory leak danger, you might want to combine those functions inside the C code.

Even if wrapping C++ classes might be a huge pain, you can still just call the whole program and hide all the templates and classes inside that.

In [9]:
lo=ctypes.CDLL('./libshapes.so')

In [10]:
lo.NOT_main()

0